In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata
from scipy.sparse import find
from scbasset.utils import *
import seaborn as sns
import matplotlib.pyplot as plt

import os

/tmp/ipykernel_9796/2271081770.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-11 09:23:57.814914: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 09:23:58.323402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 09:23:58.323643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin 

## Enhancer

In [5]:
data_path = '../../../results_scbasset/enhancer_final/data/'
type_data = 'enhancer'
#filter_info = 'filtered'
filter_info = 'not_filtered'
filter_info = 'filtered_controle'

## Promotor

In [2]:
data_path = '../../../results_scbasset/promotor_final/data/'
type_data = 'promotor'
#filter_info = 'filtered'
#filter_info = 'not_filtered'
filter_info = 'filtered_controle'

## Enhancer_promotor

In [18]:
data_path = '../../../results_scbasset/enhancer_promotor_final/data/'
type_data = 'enhancer_promotor'
#filter_info = 'filtered'
filter_info = 'not_filtered'

## SCAFE enhancer

In [2]:
data_path = '../../../results_scbasset/enhancer_SCAFE/data/'
type_data = 'enhancer'
#filter_info = 'filtered'
filter_info = 'not_filtered'

In [4]:
def Visualize_h5_file(file_name, group):
    """
    Visualize the contents of an HDF5 file.

    Parameters:
    - file_name (str): The name of the HDF5 file.
    - group (str): The name of the group within the HDF5 file.

    """
    with h5py.File(file_name, 'r') as f:
        # Retrieve the keys of the HDF5 file
        keys = list(f.keys())
        print(f"Keys available in the HDF5 file: {keys}\n")

        # Access the specified group
        group_data = f[group]

        # Iterate over each member of the group
        for member in group_data.keys():
            # Check if the member is a group or a dataset
            if isinstance(group_data[member], h5py.Group):
                # If it's a group, display its keys
                print(f"Keys of subgroup '{member}':", list(group_data[member].keys()))
            else:
                # Otherwise, display the dataset's value
                dimensions = group_data[member].shape
                print(f"Dimension: {dimensions}")
                print(f"Value of dataset '{member}':", group_data[member][:])

In [6]:
Visualize_h5_file(count_matrix_file, 'matrix')

Keys available in the HDF5 file: ['matrix']

Dimension: (104042,)
Value of dataset 'barcodes': [b'NL102A_AAAGTAGCATTCTTAC-1' b'NL102A_AACCATGGTCAACATC-1'
 b'NL102A_AACCGCGGTACCAGTT-1' ... b'NL99C_TTTGTCATCAATAAGG-1'
 b'NL99C_TTTGTCATCTATCCCG-1' b'NL99C_TTTGTCATCTCCAGGG-1']
Dimension: (21675128,)
Value of dataset 'data': [1 1 1 ... 1 1 1]
Keys of subgroup 'features': ['_all_tag_keys', 'feature_type', 'genome', 'id', 'interval', 'name']
Dimension: (21675128,)
Value of dataset 'indices': [    17    274   1342 ... 123545 129667 137382]
Dimension: (104043,)
Value of dataset 'indptr': [       0      323      786 ... 21675005 21675100 21675128]
Dimension: (2,)
Value of dataset 'shape': [153373 104042]


# Read the count_matrix and peaks

In [3]:
count_matrix_file = f'{data_path}count_matrix_{type_data}.h5'
bed_file = f'{data_path}peaks_{type_data}.bed'

In [4]:
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'])
ad = sc.read_10x_h5(count_matrix_file, gex_only=False)

In [5]:
print(f'Number of {type_data}: {ad.shape[1]} \nNumber of patient: {ad.shape[0]}')

Number of enhancer: 153373 
Number of patient: 104042


In [13]:
ad_cage.var

,gene_ids,feature_types,genome,chr,start,end,n_cells
chr1-10551-11358--,chr1-10551-11358--,Peaks,GRCh38,chr1,10550,11358,26
chr1-14569-15070--,chr1-14569-15070--,Peaks,GRCh38,chr1,14568,15070,740
chr1-16267-16768--,chr1-16267-16768--,Peaks,GRCh38,chr1,16266,16768,142
chr1-24805-25306--,chr1-24805-25306--,Peaks,GRCh38,chr1,24804,25306,10
chr1-34491-35027-+,chr1-34491-35027-+,Peaks,GRCh38,chr1,34490,35027,4
...,...,...,...,...,...,...,...
chrY-56851291-56851792-+,chrY-56851291-56851792-+,Peaks,GRCh38,chrY,56851290,56851792,3
chrY-56863540-56864041--,chrY-56863540-56864041--,Peaks,GRCh38,chrY,56863539,56864041,4
chrY-56868446-56868947--,chrY-56868446-56868947--,Peaks,GRCh38,chrY,56868445,56868947,2
chrY-56881448-56881949-+,chrY-56881448-56881949-+,Peaks,GRCh38,chrY,56881447,56881949,12


# Filtering

## Filtering on peak accessibilities in patient 

In [33]:
ad_cage = ad[:, ad.var['feature_types']=='Peaks']
ad_cage.var['chr'] = peak['chr'].values
ad_cage.var['start'] = peak['start'].values
ad_cage.var['end'] = peak['end'].values

# # basic stats
sc.pp.filter_cells(ad_cage, min_genes=0)
sc.pp.filter_genes(ad_cage, min_cells=0)

# # a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0)
ad_cage = ad_cage[:, ad_cage.var['n_cells']>=thres]

/tmp/ipykernel_80912/1540882627.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ad_cage.var['chr'] = peak['chr'].values


In [35]:
print(f'Number of {type_data}: {ad_cage.shape[1]} \nNumber of patient: {ad_cage.shape[0]}')

Number of enhancer: 153373 
Number of patient: 104042


## Filtering on chromosomes

In [37]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_cage = ad_cage[:, ad_cage.var['chr'].isin(chrs)]

In [38]:
print(f'Number of {type_data}: {ad_cage.shape[1]} \nNumber of patient: {ad_cage.shape[0]}')

Number of enhancer: 153372 
Number of patient: 104042


In [27]:
adata.var

,gene_ids,feature_types,genome,chr,start,end,n_cells
chr1-10551-11358--,chr1-10551-11358--,Peaks,GRCh38,chr1,10550,11358,26
chr1-14569-15070--,chr1-14569-15070--,Peaks,GRCh38,chr1,14568,15070,740
chr1-16267-16768--,chr1-16267-16768--,Peaks,GRCh38,chr1,16266,16768,142
chr1-24805-25306--,chr1-24805-25306--,Peaks,GRCh38,chr1,24804,25306,10
chr1-34491-35027-+,chr1-34491-35027-+,Peaks,GRCh38,chr1,34490,35027,4
...,...,...,...,...,...,...,...
chrY-56851291-56851792-+,chrY-56851291-56851792-+,Peaks,GRCh38,chrY,56851290,56851792,3
chrY-56863540-56864041--,chrY-56863540-56864041--,Peaks,GRCh38,chrY,56863539,56864041,4
chrY-56868446-56868947--,chrY-56868446-56868947--,Peaks,GRCh38,chrY,56868445,56868947,2
chrY-56881448-56881949-+,chrY-56881448-56881949-+,Peaks,GRCh38,chrY,56881447,56881949,12


### save h5ad

In [39]:
ad_cage.write(f'{data_path}{filter_info}/cage_{type_data}_ad.h5ad')

/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
